# Import Relevant Libraries and Load in Merged Datasets

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,MaxAbsScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import numpy as np
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('data/merged_data.csv')

/Users/jamiecarnevale/miniconda3/envs/learn-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (10,21,30,63,97,99,118,119,120,122,126,127,128,131,133,136,137,139,149) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


# Data Cleaning

In [4]:
df.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 150 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   SEAT_NO                        60155 non-null   float64
 8   CITY                           460748 non-null  object 
 9   STATE                          459075 non-null  object 
 10  ZIPCODE                        422795 non-null  object 
 11  SEX                            727658 non-null  object 
 12  AGE                          

In [5]:
df.iloc[:,100:120].head()

,TOWED_BY,TOWED_TO,AREA_00_I,AREA_01_I,AREA_02_I,AREA_03_I,AREA_04_I,AREA_05_I,AREA_06_I,AREA_07_I,AREA_08_I,AREA_09_I,AREA_10_I,AREA_11_I,AREA_12_I,AREA_99_I,FIRST_CONTACT_POINT,CMV_ID,USDOT_NO,CCMC_NO
0,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FRONT,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SIDE-RIGHT-FRONT,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,REAR,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,NaN,NaN,FRONT-LEFT,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,UNKNOWN,NaN,NaN,NaN


### let's drop all columns that have less than 20k nonnull values

In [6]:
df.columns[df.notna().sum() < 20000]

Index(['EMS_RUN_NO', 'PEDPEDAL_ACTION', 'PEDPEDAL_VISIBILITY',
       'PEDPEDAL_LOCATION', 'BAC_RESULT VALUE', 'CELL_PHONE_USE',
       'PHOTOS_TAKEN_I', 'STATEMENTS_TAKEN_I', 'DOORING_I', 'WORK_ZONE_I',
       'WORK_ZONE_TYPE', 'WORKERS_PRESENT_I', 'CMRC_VEH_I', 'FIRE_I',
       'EXCEED_SPEED_LIMIT_I', 'CMV_ID', 'USDOT_NO', 'CCMC_NO', 'ILCC_NO',
       'COMMERCIAL_SRC', 'GVWR', 'CARRIER_NAME', 'CARRIER_STATE',
       'CARRIER_CITY', 'HAZMAT_PLACARDS_I', 'HAZMAT_NAME', 'UN_NO',
       'HAZMAT_PRESENT_I', 'HAZMAT_REPORT_I', 'HAZMAT_REPORT_NO',
       'MCS_REPORT_I', 'MCS_REPORT_NO', 'HAZMAT_VIO_CAUSE_CRASH_I',
       'MCS_VIO_CAUSE_CRASH_I', 'IDOT_PERMIT_NO', 'WIDE_LOAD_I',
       'TRAILER1_WIDTH', 'TRAILER2_WIDTH', 'TRAILER1_LENGTH',
       'TRAILER2_LENGTH', 'TOTAL_VEHICLE_LENGTH', 'AXLE_CNT', 'VEHICLE_CONFIG',
       'CARGO_BODY_TYPE', 'LOAD_TYPE', 'HAZMAT_OUT_OF_SERVICE_I',
       'MCS_OUT_OF_SERVICE_I', 'HAZMAT_CLASS'],
      dtype='object')

In [7]:
# dropping all columns that have less than 250k non null values
colstodrop = df.columns[df.notna().sum() < 250000]

In [8]:
df1 = df.drop(colstodrop, axis = 1)

In [9]:
df1['PERSON_TYPE'].value_counts()

DRIVER                 665215
PASSENGER               60155
BICYCLE                  3524
PEDESTRIAN               2403
NON-MOTOR VEHICLE         464
NON-CONTACT VEHICLE       146
Name: PERSON_TYPE, dtype: int64

In [10]:
df1

,Unnamed: 0,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO_x,VEHICLE_ID_x,CRASH_DATE_x,CITY,STATE,ZIPCODE,...,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT
0,0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,CHICAGO,IL,60651,...,CIVIC,IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,FRONT
1,1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,CHICAGO,IL,60620,...,ENCORE,IL,NaN,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1.0,SIDE-RIGHT-FRONT
2,2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,NONE,PASSENGER,PERSONAL,E,BACKING,1.0,REAR
3,3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,...,GRAND CARAVAN,IL,2012.0,NONE,PASSENGER,NOT IN USE,N,PARKED,0.0,FRONT-LEFT
4,4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,UNKNOWN/NA,1.0,UNKNOWN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731902,731902,O1559700,DRIVER,462b5811f1ff74d554aaaa269233f41a68635bb3d1e52c...,JG228070,1483547.0,04/18/2023 11:32:00 AM,ROUND LAKE,IL,60073,...,PATRIOT,IL,2015.0,UNKNOWN,PASSENGER,PERSONAL,S,TURNING ON RED,2.0,SIDE-RIGHT
731903,731903,O1560774,DRIVER,dc203cc2a56651a63090b1817fbd40e8905efa97031346...,JG234213,1484605.0,04/18/2023 06:45:00 AM,NaN,NaN,NaN,...,DURANGO,IL,2017.0,UNKNOWN,SPORT UTILITY VEHICLE (SUV),PERSONAL,S,PASSING/OVERTAKING,1.0,FRONT-LEFT-CORNER
731904,731904,O1561819,DRIVER,aa8ada359fd31d545b59706ded379ea1193d43419f76e3...,JG236026,1485616.0,04/18/2023 07:12:00 PM,NaN,NaN,NaN,...,OTHER (EXPLAIN IN NARRATIVE),IL,2005.0,NONE,PICKUP,OTHER,E,STRAIGHT AHEAD,1.0,FRONT-RIGHT-CORNER
731905,731905,O1563799,DRIVER,4050f4309ea3c72d42fbe9b22bb9933d9268c543147e3a...,JG228786,1487508.0,04/18/2023 06:20:00 PM,CHICAGO,IL,60616,...,RAV4,IL,2018.0,UNKNOWN,SPORT UTILITY VEHICLE (SUV),PERSONAL,W,TURNING RIGHT,1.0,FRONT-RIGHT-CORNER


In [11]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 76 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   CITY                           460748 non-null  object 
 8   STATE                          459075 non-null  object 
 9   ZIPCODE                        422795 non-null  object 
 10  SEX                            727658 non-null  object 
 11  AGE                            434077 non-null  float64
 12  DRIVERS_LICENSE_STATE         

In [12]:
df1.head()

,Unnamed: 0,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,RD_NO_x,VEHICLE_ID_x,CRASH_DATE_x,CITY,STATE,ZIPCODE,...,MODEL,LIC_PLATE_STATE,VEHICLE_YEAR,VEHICLE_DEFECT,VEHICLE_TYPE,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT
0,0,O749947,DRIVER,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,834816.0,09/28/2019 03:30:00 AM,CHICAGO,IL,60651,...,CIVIC,IL,2016.0,UNKNOWN,PASSENGER,PERSONAL,N,STRAIGHT AHEAD,1.0,FRONT
1,1,O871921,DRIVER,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,827212.0,04/13/2020 10:50:00 PM,CHICAGO,IL,60620,...,ENCORE,IL,NaN,NONE,PASSENGER,PERSONAL,W,STRAIGHT AHEAD,1.0,SIDE-RIGHT-FRONT
2,2,O10018,DRIVER,71162af7bf22799b776547132ebf134b5b438dcf3dac6b...,HY484534,9579.0,11/01/2015 05:00:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,NONE,PASSENGER,PERSONAL,E,BACKING,1.0,REAR
3,3,O10038,DRIVER,c21c476e2ccc41af550b5d858d22aaac4ffc88745a1700...,HY484750,9598.0,11/01/2015 08:00:00 AM,NaN,NaN,NaN,...,GRAND CARAVAN,IL,2012.0,NONE,PASSENGER,NOT IN USE,N,PARKED,0.0,FRONT-LEFT
4,4,O10039,DRIVER,eb390a4c8e114c69488f5fb8a097fe629f5a92fd528cf4...,HY484778,9600.0,11/01/2015 10:15:00 AM,NaN,NaN,NaN,...,UNKNOWN,XX,NaN,UNKNOWN,UNKNOWN/NA,UNKNOWN/NA,UNKNOWN,UNKNOWN/NA,1.0,UNKNOWN


In [13]:
df1['DRIVER_VISION'].value_counts()

UNKNOWN                   405885
NOT OBSCURED              245541
OTHER                       8286
MOVING VEHICLES             4297
PARKED VEHICLES             3043
WINDSHIELD (WATER/ICE)      1894
BLINDED - SUNLIGHT          1261
TREES, PLANTS                340
BUILDINGS                    258
BLINDED - HEADLIGHTS          89
BLOWING MATERIALS             63
HILLCREST                     61
EMBANKMENT                    44
SIGNBOARD                     20
Name: DRIVER_VISION, dtype: int64

In [14]:
df1['MOST_SEVERE_INJURY'].value_counts()

NO INDICATION OF INJURY     632576
NONINCAPACITATING INJURY     56080
REPORTED, NOT EVIDENT        29971
INCAPACITATING INJURY        12470
FATAL                          799
Name: MOST_SEVERE_INJURY, dtype: int64

In [15]:
df1['VEHICLE_DEFECT'].value_counts()

UNKNOWN             398828
NONE                305305
OTHER                 4822
BRAKES                3524
TIRES                  632
STEERING               597
WHEELS                 329
SUSPENSION             222
ENGINE/MOTOR           129
FUEL SYSTEM            117
LIGHTS                  59
WINDOWS                 52
CARGO                   35
SIGNALS                 22
RESTRAINT SYSTEM        17
TRAILER COUPLING        15
EXHAUST                 11
Name: VEHICLE_DEFECT, dtype: int64

In [16]:
df1['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 282207
FAILING TO YIELD RIGHT-OF-WAY                                                        80260
FOLLOWING TOO CLOSELY                                                                72567
NOT APPLICABLE                                                                       38452
IMPROPER OVERTAKING/PASSING                                                          35747
FAILING TO REDUCE SPEED TO AVOID CRASH                                               31048
IMPROPER BACKING                                                                     29473
IMPROPER LANE USAGE                                                                  26774
IMPROPER TURNING/NO SIGNAL                                                           24259
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  24041
DISREGARDING TRAFFIC SIGNALS                                                         14223

In [17]:
df1['LOCATION'].value_counts()

POINT (-87.905309125103 41.976201139024)    1161
POINT (-87.619928173678 41.900958919109)     674
POINT (-87.580147768689 41.791420282098)     527
POINT (-87.585971992965 41.751460603167)     512
POINT (-87.585275565077 41.722257273006)     400
                                            ... 
POINT (-87.675500965327 41.772165226802)       1
POINT (-87.618854662228 41.760054855524)       1
POINT (-87.583302651416 41.747754835025)       1
POINT (-87.796635332255 41.968860630424)       1
POINT (-87.719740441587 41.69846844971)        1
Name: LOCATION, Length: 279315, dtype: int64

In [18]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 76 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   Unnamed: 0                     731907 non-null  int64  
 1   PERSON_ID                      731907 non-null  object 
 2   PERSON_TYPE                    731907 non-null  object 
 3   CRASH_RECORD_ID                731907 non-null  object 
 4   RD_NO_x                        727570 non-null  object 
 5   VEHICLE_ID_x                   725151 non-null  float64
 6   CRASH_DATE_x                   731907 non-null  object 
 7   CITY                           460748 non-null  object 
 8   STATE                          459075 non-null  object 
 9   ZIPCODE                        422795 non-null  object 
 10  SEX                            727658 non-null  object 
 11  AGE                            434077 non-null  float64
 12  DRIVERS_LICENSE_STATE         

### lets further hone in on the relevant columns

In [19]:
df2 = df1[[
    'PERSON_TYPE',
    'SEX',
    'EJECTION',
    'INJURY_CLASSIFICATION',
    'DRIVER_VISION',
    'PHYSICAL_CONDITION',
    'POSTED_SPEED_LIMIT',
    'TRAFFIC_CONTROL_DEVICE',
    'DEVICE_CONDITION',
    'WEATHER_CONDITION',
    'LIGHTING_CONDITION',
    'FIRST_CRASH_TYPE',
    'TRAFFICWAY_TYPE',
    'ALIGNMENT',
    'ROADWAY_SURFACE_COND',
    'ROAD_DEFECT',
    'REPORT_TYPE',
    'CRASH_TYPE',
    'DAMAGE',
    'DATE_POLICE_NOTIFIED',
    'PRIM_CONTRIBUTORY_CAUSE',
    'STREET_NO',
    'STREET_DIRECTION',
    'STREET_NAME',
    'BEAT_OF_OCCURRENCE',
    'NUM_UNITS',
    'MOST_SEVERE_INJURY',
    'INJURIES_TOTAL',
    'INJURIES_FATAL',
    'INJURIES_INCAPACITATING',
    'INJURIES_NON_INCAPACITATING',
    'INJURIES_REPORTED_NOT_EVIDENT',
    'INJURIES_NO_INDICATION',
    'INJURIES_UNKNOWN',
    'CRASH_HOUR',
    'CRASH_DAY_OF_WEEK',
    'CRASH_MONTH',
    'LATITUDE',
    'LONGITUDE',
    'LOCATION',
    'CRASH_DATE',
    'UNIT_TYPE',
    'MAKE',
    'MODEL',
    'LIC_PLATE_STATE',
    'VEHICLE_YEAR',
    'VEHICLE_DEFECT',
    'VEHICLE_TYPE',
    'VEHICLE_USE',
    'TRAVEL_DIRECTION',
    'MANEUVER',
    'OCCUPANT_CNT',
    'FIRST_CONTACT_POINT',
    'AGE',
    'BAC_RESULT',
    'SAFETY_EQUIPMENT',
    'AIRBAG_DEPLOYED'
]]



In [20]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 731907 entries, 0 to 731906
Data columns (total 57 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    731907 non-null  object 
 1   SEX                            727658 non-null  object 
 2   EJECTION                       729172 non-null  object 
 3   INJURY_CLASSIFICATION          731723 non-null  object 
 4   DRIVER_VISION                  671082 non-null  object 
 5   PHYSICAL_CONDITION             671290 non-null  object 
 6   POSTED_SPEED_LIMIT             731907 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         731907 non-null  object 
 8   DEVICE_CONDITION               731907 non-null  object 
 9   WEATHER_CONDITION              731907 non-null  object 
 10  LIGHTING_CONDITION             731907 non-null  object 
 11  FIRST_CRASH_TYPE               731907 non-null  object 
 12  TRAFFICWAY_TYPE               

## lets do some quick cleaning up of outliers

In [21]:
df2['VEHICLE_YEAR'].value_counts()
#looks like there are some vehicle years that dont make sense

2015.0    36281
2016.0    33811
2014.0    32851
2017.0    32795
2013.0    30706
          ...  
3016.0        1
3023.0        1
3027.0        1
5004.0        1
2054.0        1
Name: VEHICLE_YEAR, Length: 169, dtype: int64

In [22]:
#quickly cleaning up the outliers
df2 = df2[(df['VEHICLE_YEAR'] <= 2024) & (df['VEHICLE_YEAR'] >= 1930)]

In [23]:
df2['VEHICLE_YEAR'].value_counts().tail(20)
#looks clean

1977.0    16
1971.0    14
1974.0    11
1969.0    11
1973.0     9
1965.0     6
1964.0     6
1963.0     5
1966.0     5
1968.0     5
1967.0     3
1953.0     2
1952.0     2
1962.0     1
1961.0     1
1960.0     1
1958.0     1
1957.0     1
1954.0     1
1941.0     1
Name: VEHICLE_YEAR, dtype: int64

In [24]:
df2['AGE'].value_counts().tail()
#need to clean up some of these negative values that can't exist

 109.0    2
 102.0    2
-1.0      1
 110.0    1
 108.0    1
Name: AGE, dtype: int64

In [25]:
df2 = df2[(df['AGE'] <= 110) & (df['AGE'] >= 0)]
#this should do it

<ipython-input-25-2ba7b13e0de3>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df2 = df2[(df['AGE'] <= 110) & (df['AGE'] >= 0)]


In [26]:
df2['AGE'].value_counts().tail()
#better

107.0    3
102.0    2
109.0    2
108.0    1
110.0    1
Name: AGE, dtype: int64

In [27]:
df2['CRASH_DAY_OF_WEEK'].value_counts()
#looks clean already

6    65469
5    58372
4    57633
3    57587
7    56390
2    54284
1    45259
Name: CRASH_DAY_OF_WEEK, dtype: int64

In [28]:
df2.groupby('MOST_SEVERE_INJURY').mean()
#just curious

,POSTED_SPEED_LIMIT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,VEHICLE_YEAR,OCCUPANT_CNT,AGE
MOST_SEVERE_INJURY,,,,,,,,,,,,,,,,,,,
FATAL,29.818913,4334.507042,1041.156942,2.062374,1.937626,1.096579,0.287726,0.460765,0.092555,0.730382,0.0,12.156942,4.054326,6.692153,41.823726,-87.670369,2008.941650,1.593561,38.066398
INCAPACITATING INJURY,30.046626,3850.259166,1158.003985,2.138417,1.643597,0.000000,1.206562,0.341259,0.095776,1.193544,0.0,12.792508,4.081031,6.733927,41.844281,-87.665615,2010.058980,1.461211,37.642667
NO INDICATION OF INJURY,28.491539,3618.968111,1276.278245,2.004651,0.000000,0.000000,0.000000,0.000000,0.000000,2.241901,0.0,13.314517,4.150524,6.544793,41.860576,-87.674103,2010.644449,1.181486,38.923103
NONINCAPACITATING INJURY,29.898719,3808.405324,1175.471663,2.126530,1.466888,0.000000,0.000000,1.381534,0.085354,1.365092,0.0,12.972521,4.096163,6.686463,41.842549,-87.666270,2010.441639,1.409842,37.185148
"REPORTED, NOT EVIDENT",29.764955,3757.728205,1213.592640,2.121427,1.349275,0.000000,0.000000,0.000000,1.349275,1.582227,0.0,13.254900,4.128267,6.627909,41.842563,-87.664758,2010.460698,1.364434,37.388628


In [29]:
df2.columns

Index(['PERSON_TYPE', 'SEX', 'EJECTION', 'INJURY_CLASSIFICATION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE',
       'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION',
       'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'CRASH_DATE', 'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRE

In [30]:
df2['MAKE'].value_counts()

CHEVROLET                       49842
FORD                            47011
NISSAN                          33041
HONDA                           30642
TOYOTA MOTOR COMPANY, LTD.      26856
                                ...  
MONO                                1
BALDWIN ENTERPRISES                 1
SAFTI                               1
TCI, INC., BENSON, MINNESOTA        1
PLATT TRAILER COMPANY               1
Name: MAKE, Length: 794, dtype: int64

In [31]:
df2['BAC_RESULT'].value_counts()

TEST NOT OFFERED                   354396
TEST REFUSED                         5232
TEST TAKEN                           1686
TEST PERFORMED, RESULTS UNKNOWN      1474
Name: BAC_RESULT, dtype: int64

In [32]:
df2.shape

(394994, 57)

# Creating Target Prediction Column

In [33]:
df2['PRIM_CONTRIBUTORY_CAUSE'].value_counts()

UNABLE TO DETERMINE                                                                 120310
FAILING TO YIELD RIGHT-OF-WAY                                                        54067
FOLLOWING TOO CLOSELY                                                                44921
FAILING TO REDUCE SPEED TO AVOID CRASH                                               20164
IMPROPER OVERTAKING/PASSING                                                          17886
NOT APPLICABLE                                                                       17128
IMPROPER BACKING                                                                     16356
IMPROPER TURNING/NO SIGNAL                                                           15945
IMPROPER LANE USAGE                                                                  15238
DRIVING SKILLS/KNOWLEDGE/EXPERIENCE                                                  14802
DISREGARDING TRAFFIC SIGNALS                                                          8834

In [34]:
len(df2.loc[(df2['PRIM_CONTRIBUTORY_CAUSE'] =='UNABLE TO DETERMINE') | (df2['PRIM_CONTRIBUTORY_CAUSE'] == 'NOT APPLICABLE')])/df2.shape[0]

0.34794958910768264

In [35]:
target_dict = {
    'reckless driving': [
        'FOLLOWING TOO CLOSELY', 
        'IMPROPER OVERTAKING/PASSING', 
        'EXCEEDING AUTHORIZED SPEED LIMIT', 
        'IMPROPER TURNING/NO SIGNAL', 
        'IMPROPER BACKING',  
        'IMPROPER LANE USAGE', 
        'FAILING TO REDUCE SPEED TO AVOID CRASH', 
        'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
        'TURNING RIGHT ON RED', 
        'DRIVING ON WRONG SIDE/WRONG WAY',
        'CELL PHONE USE OTHER THAN TEXTING', 
        'TEXTING',
        'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)', 
        'DISTRACTION - FROM INSIDE VEHICLE'
    ],

    'missed or misunderstood traffic signs': [
        'DISREGARDING YIELD SIGN', 
        'FAILING TO YIELD RIGHT-OF-WAY', 
        'DISREGARDING TRAFFIC SIGNALS', 
        'DISREGARDING OTHER TRAFFIC SIGNS',
        'DISREGARDING ROAD MARKINGS', 
        'DISREGARDING STOP SIGN'
    ],

    'issue with the driver or vehicle': [
        'DRIVING SKILLS/KNOWLEDGE/EXPERIENCE', 
        'PHYSICAL CONDITION OF DRIVER', 
        'EQUIPMENT - VEHICLE CONDITION', 
        'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',
        'UNDER THE INFLUENCE OF ALCOHOL/DRUGS (USE WHEN ARREST IS EFFECTED)'
    ],

    'external variable': [
        'WEATHER', 
        'DISTRACTION - FROM OUTSIDE VEHICLE',
        'VISION OBSCURED (SIGNS, TREE LIMBS, BUILDINGS, ETC.)', 
        'EVASIVE ACTION DUE TO ANIMAL, OBJECT, NONMOTORIST',
        'ROAD CONSTRUCTION/MAINTENANCE', 
        'ROAD ENGINEERING/SURFACE/MARKING DEFECTS', 
        'ANIMAL', 
        'OBSTRUCTED CROSSWALKS',
        'EXCEEDING SAFE SPEED FOR CONDITIONS',
    ],

    'drop': [
        'PASSING STOPPED SCHOOL BUS', 
        'MOTORCYCLE ADVANCING LEGALLY ON RED LIGHT',
        'BICYCLE ADVANCING LEGALLY ON RED LIGHT', 
        'RELATED TO BUS STOP'
    ]
}


In [36]:
target_dict

{'reckless driving': ['FOLLOWING TOO CLOSELY',
  'IMPROPER OVERTAKING/PASSING',
  'EXCEEDING AUTHORIZED SPEED LIMIT',
  'IMPROPER TURNING/NO SIGNAL',
  'IMPROPER BACKING',
  'IMPROPER LANE USAGE',
  'FAILING TO REDUCE SPEED TO AVOID CRASH',
  'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
  'TURNING RIGHT ON RED',
  'DRIVING ON WRONG SIDE/WRONG WAY',
  'CELL PHONE USE OTHER THAN TEXTING',
  'TEXTING',
  'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',
  'DISTRACTION - FROM INSIDE VEHICLE'],
 'missed or misunderstood traffic signs': ['DISREGARDING YIELD SIGN',
  'FAILING TO YIELD RIGHT-OF-WAY',
  'DISREGARDING TRAFFIC SIGNALS',
  'DISREGARDING OTHER TRAFFIC SIGNS',
  'DISREGARDING ROAD MARKINGS',
  'DISREGARDING STOP SIGN'],
 'issue with the driver or vehicle': ['DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
  'PHYSICAL CONDITION OF DRIVER',
  'EQUIPMENT - VEHICLE CONDITION',
  'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',


In [37]:
new_dict = {value: key for key in target_dict for value in target_dict[key]}

In [38]:
df2['target_mapped'] = df2['PRIM_CONTRIBUTORY_CAUSE']

In [39]:
df2['target_mapped'] = df2['target_mapped'].map(new_dict)

In [40]:
df2['target_mapped'].value_counts()

reckless driving                         142300
missed or misunderstood traffic signs     68788
issue with the driver or vehicle          26041
external variable                         20178
drop                                        249
Name: target_mapped, dtype: int64

In [41]:
df2 = df2[df2['target_mapped'] != 'drop']

# Further Cleaning (including dropping null values)

In [42]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 394745 entries, 0 to 731905
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    394745 non-null  object 
 1   SEX                            394396 non-null  object 
 2   EJECTION                       394669 non-null  object 
 3   INJURY_CLASSIFICATION          394742 non-null  object 
 4   DRIVER_VISION                  362568 non-null  object 
 5   PHYSICAL_CONDITION             362572 non-null  object 
 6   POSTED_SPEED_LIMIT             394745 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         394745 non-null  object 
 8   DEVICE_CONDITION               394745 non-null  object 
 9   WEATHER_CONDITION              394745 non-null  object 
 10  LIGHTING_CONDITION             394745 non-null  object 
 11  FIRST_CRASH_TYPE               394745 non-null  object 
 12  TRAFFICWAY_TYPE               

In [43]:
df2 = df2.dropna()

In [44]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218191 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    218191 non-null  object 
 1   SEX                            218191 non-null  object 
 2   EJECTION                       218191 non-null  object 
 3   INJURY_CLASSIFICATION          218191 non-null  object 
 4   DRIVER_VISION                  218191 non-null  object 
 5   PHYSICAL_CONDITION             218191 non-null  object 
 6   POSTED_SPEED_LIMIT             218191 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         218191 non-null  object 
 8   DEVICE_CONDITION               218191 non-null  object 
 9   WEATHER_CONDITION              218191 non-null  object 
 10  LIGHTING_CONDITION             218191 non-null  object 
 11  FIRST_CRASH_TYPE               218191 non-null  object 
 12  TRAFFICWAY_TYPE              

In [45]:
df2.head()

,PERSON_TYPE,SEX,EJECTION,INJURY_CLASSIFICATION,DRIVER_VISION,PHYSICAL_CONDITION,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,...,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT,AGE,BAC_RESULT,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,target_mapped
12,DRIVER,F,NONE,NO INDICATION OF INJURY,NOT OBSCURED,NORMAL,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,...,PERSONAL,NE,MERGING,1.0,FRONT-LEFT-CORNER,34.0,TEST NOT OFFERED,SAFETY BELT USED,DID NOT DEPLOY,reckless driving
18,DRIVER,M,NONE,NO INDICATION OF INJURY,NOT OBSCURED,UNKNOWN,15,NO CONTROLS,NO CONTROLS,CLEAR,...,PERSONAL,W,STRAIGHT AHEAD,1.0,FRONT-RIGHT-CORNER,22.0,TEST NOT OFFERED,SAFETY BELT USED,DID NOT DEPLOY,reckless driving
37,DRIVER,M,NONE,NO INDICATION OF INJURY,UNKNOWN,NORMAL,30,NO CONTROLS,NO CONTROLS,CLEAR,...,PERSONAL,N,PASSING/OVERTAKING,1.0,SIDE-RIGHT-FRONT,22.0,TEST NOT OFFERED,USAGE UNKNOWN,"DEPLOYED, FRONT",reckless driving
38,DRIVER,M,NONE,NO INDICATION OF INJURY,NOT OBSCURED,NORMAL,45,NO CONTROLS,NO CONTROLS,CLEAR,...,PERSONAL,N,SLOW/STOP IN TRAFFIC,2.0,REAR,61.0,TEST NOT OFFERED,SAFETY BELT USED,DID NOT DEPLOY,reckless driving
42,DRIVER,M,NONE,NO INDICATION OF INJURY,NOT OBSCURED,NORMAL,35,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,...,PERSONAL,N,TURNING LEFT,1.0,FRONT-LEFT-CORNER,28.0,TEST NOT OFFERED,SAFETY BELT USED,"DEPLOYED, COMBINATION",missed or misunderstood traffic signs


In [46]:
for column in df2.columns:
    print(f"Column: {column}")
    print(df2[column].value_counts())
    print("\n")


Column: PERSON_TYPE
DRIVER                 218178
NON-CONTACT VEHICLE        11
NON-MOTOR VEHICLE           2
Name: PERSON_TYPE, dtype: int64


Column: SEX
M    136585
F     81382
X       224
Name: SEX, dtype: int64


Column: EJECTION
NONE                  213827
UNKNOWN                 3492
TOTALLY EJECTED          562
PARTIALLY EJECTED        163
TRAPPED/EXTRICATED       147
Name: EJECTION, dtype: int64


Column: INJURY_CLASSIFICATION
NO INDICATION OF INJURY     202568
NONINCAPACITATING INJURY      9455
REPORTED, NOT EVIDENT         4197
INCAPACITATING INJURY         1858
FATAL                          113
Name: INJURY_CLASSIFICATION, dtype: int64


Column: DRIVER_VISION
NOT OBSCURED              132546
UNKNOWN                    73567
OTHER                       4643
MOVING VEHICLES             2851
PARKED VEHICLES             2094
WINDSHIELD (WATER/ICE)      1181
BLINDED - SUNLIGHT           774
TREES, PLANTS                216
BUILDINGS                    159
BLINDED - HEADLIGHTS 

1834.0    2973
114.0     2722
122.0     2408
1831.0    2171
1622.0    1954
          ... 
232.0      277
221.0      271
1114.0     268
422.0      190
1125.0     157
Name: BEAT_OF_OCCURRENCE, Length: 271, dtype: int64


Column: NUM_UNITS
2     187075
3      13826
1      13643
4       2715
5        608
6        198
7         63
8         34
9         10
10         9
11         4
12         3
18         2
15         1
Name: NUM_UNITS, dtype: int64


Column: MOST_SEVERE_INJURY
NO INDICATION OF INJURY     182524
NONINCAPACITATING INJURY     20197
REPORTED, NOT EVIDENT        10991
INCAPACITATING INJURY         4278
FATAL                          201
Name: MOST_SEVERE_INJURY, dtype: int64


Column: INJURIES_TOTAL
0.0     182524
1.0      25834
2.0       6823
3.0       1905
4.0        694
5.0        257
6.0         98
7.0         25
8.0         15
9.0          6
11.0         4
10.0         2
12.0         1
21.0         1
15.0         1
19.0         1
Name: INJURIES_TOTAL, dtype: int64


Column

STRAIGHT AHEAD                        109884
TURNING LEFT                           24342
BACKING                                16277
TURNING RIGHT                          11344
PASSING/OVERTAKING                      9688
CHANGING LANES                          8791
SLOW/STOP IN TRAFFIC                    6486
ENTERING TRAFFIC LANE FROM PARKING      4962
OTHER                                   3624
U-TURN                                  3067
SKIDDING/CONTROL LOSS                   2822
MERGING                                 2815
UNKNOWN/NA                              2705
AVOIDING VEHICLES/OBJECTS               2127
PARKED                                  1793
STARTING IN TRAFFIC                     1770
ENTER FROM DRIVE/ALLEY                  1757
LEAVING TRAFFIC LANE TO PARK            1461
DRIVING WRONG WAY                        662
NEGOTIATING A CURVE                      518
SLOW/STOP - LEFT TURN                    439
SLOW/STOP - RIGHT TURN                   313
SLOW/STOP 

In [47]:
df2['DAMAGE'].value_counts()

OVER $1,500      144951
$501 - $1,500     51963
$500 OR LESS      21277
Name: DAMAGE, dtype: int64

In [48]:
df2.groupby('MOST_SEVERE_INJURY').count()

,PERSON_TYPE,SEX,EJECTION,INJURY_CLASSIFICATION,DRIVER_VISION,PHYSICAL_CONDITION,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,...,VEHICLE_USE,TRAVEL_DIRECTION,MANEUVER,OCCUPANT_CNT,FIRST_CONTACT_POINT,AGE,BAC_RESULT,SAFETY_EQUIPMENT,AIRBAG_DEPLOYED,target_mapped
MOST_SEVERE_INJURY,,,,,,,,,,,,,,,,,,,,,
FATAL,201,201,201,201,201,201,201,201,201,201,...,201,201,201,201,201,201,201,201,201,201
INCAPACITATING INJURY,4278,4278,4278,4278,4278,4278,4278,4278,4278,4278,...,4278,4278,4278,4278,4278,4278,4278,4278,4278,4278
NO INDICATION OF INJURY,182524,182524,182524,182524,182524,182524,182524,182524,182524,182524,...,182524,182524,182524,182524,182524,182524,182524,182524,182524,182524
NONINCAPACITATING INJURY,20197,20197,20197,20197,20197,20197,20197,20197,20197,20197,...,20197,20197,20197,20197,20197,20197,20197,20197,20197,20197
"REPORTED, NOT EVIDENT",10991,10991,10991,10991,10991,10991,10991,10991,10991,10991,...,10991,10991,10991,10991,10991,10991,10991,10991,10991,10991


In [49]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218191 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    218191 non-null  object 
 1   SEX                            218191 non-null  object 
 2   EJECTION                       218191 non-null  object 
 3   INJURY_CLASSIFICATION          218191 non-null  object 
 4   DRIVER_VISION                  218191 non-null  object 
 5   PHYSICAL_CONDITION             218191 non-null  object 
 6   POSTED_SPEED_LIMIT             218191 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         218191 non-null  object 
 8   DEVICE_CONDITION               218191 non-null  object 
 9   WEATHER_CONDITION              218191 non-null  object 
 10  LIGHTING_CONDITION             218191 non-null  object 
 11  FIRST_CRASH_TYPE               218191 non-null  object 
 12  TRAFFICWAY_TYPE              

# cleaning up the make and vehicle type

In [50]:
make_counts = df2['MAKE'].value_counts()
over_50 = make_counts[make_counts > 50].index
df2 = df2[df2['MAKE'].isin(over_50)]

In [51]:
df2['MAKE'].value_counts()

CHEVROLET                            27643
FORD                                 25524
NISSAN                               18481
HONDA                                17353
TOYOTA MOTOR COMPANY, LTD.           14910
                                     ...  
INTERNATIONAL COACH MANUFACTURING       56
MOTOR COACH INDUSTRIES, INC.            55
MASERATI                                55
ICL CARGO TRAILER                       52
BLUE BIRD BODY CO.                      52
Name: MAKE, Length: 77, dtype: int64

In [52]:
df2['MAKE'] = df2['MAKE'].replace({'TOYOTA MOTOR COMPANY, LTD.' : 'TOYOTA' , 
                 'KIA MOTORS CORP' : 'KIA',
                'GENERAL MOTORS CORPORATION (GMC)': "GENERAL MOTORS CORP.",
                'TESLA MOTORS': 'TESLA',
                'LINCOLN-CONTINENTAL':'LINCOLN',
                'INTERNATIONAL TRAILER CORP.': 'INTERNATIONAL TRAILER CORPORATION',
                'PETERBILT MOTORS CO., DIVISION PACCAR, INC.' : 'PETERBILT MOTORS COMPANY (DIVISION OF PACCAR, INC.)',
                'FREIGHTLINER CORP.' : 'FREIGHTLINER CORPORATION',
                'ACURA (DIV. OF AMERICAN HONDA MOTOR CO.)' : 'ACURA',
                'KENWORTH MOTOR TRUCK CO.' : 'KENWORTH MOTOR TRUCK COMPANY'
                
                                  })

In [53]:
df2['VEHICLE_TYPE'].value_counts()

PASSENGER                                 147477
SPORT UTILITY VEHICLE (SUV)                31226
VAN/MINI-VAN                               11997
PICKUP                                      7949
TRUCK - SINGLE UNIT                         5504
UNKNOWN/NA                                  3245
TRACTOR W/ SEMI-TRAILER                     2825
OTHER                                       2105
BUS OVER 15 PASS.                           1760
BUS UP TO 15 PASS.                           516
MOTORCYCLE (OVER 150CC)                      449
TRACTOR W/O SEMI-TRAILER                     326
SINGLE UNIT TRUCK WITH TRAILER               311
OTHER VEHICLE WITH TRAILER                   278
MOTOR DRIVEN CYCLE                            27
MOPED OR MOTORIZED BICYCLE                    26
AUTOCYCLE                                     20
ALL-TERRAIN VEHICLE (ATV)                     11
3-WHEELED MOTORCYCLE (2 REAR WHEELS)           3
FARM EQUIPMENT                                 3
RECREATIONAL OFF-HIG

In [54]:
VT_counts = df2['VEHICLE_TYPE'].value_counts()
over_20 = VT_counts[VT_counts > 20].index
df2 = df2[df2['VEHICLE_TYPE'].isin(over_20)]

In [55]:
df2['VEHICLE_TYPE'].value_counts()

PASSENGER                         147477
SPORT UTILITY VEHICLE (SUV)        31226
VAN/MINI-VAN                       11997
PICKUP                              7949
TRUCK - SINGLE UNIT                 5504
UNKNOWN/NA                          3245
TRACTOR W/ SEMI-TRAILER             2825
OTHER                               2105
BUS OVER 15 PASS.                   1760
BUS UP TO 15 PASS.                   516
MOTORCYCLE (OVER 150CC)              449
TRACTOR W/O SEMI-TRAILER             326
SINGLE UNIT TRUCK WITH TRAILER       311
OTHER VEHICLE WITH TRAILER           278
MOTOR DRIVEN CYCLE                    27
MOPED OR MOTORIZED BICYCLE            26
Name: VEHICLE_TYPE, dtype: int64

## Now to baseline modeling

In [56]:
#numeric_pipeline = Pipeline([('numnorm', StandardScaler())])

#ordinal_pipeline = Pipeline([
  #  ('ordenc', OrdinalEncoder(categories = [columns])),
  #  ('ordnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(sparse = False, handle_unknown = 'ignore')), 
    ('onehotnorm', MaxAbsScaler())])


In [57]:
cat_cols = ['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE']

In [58]:
ct = ColumnTransformer(
    [ ("nominalpipe", nominal_pipeline, cat_cols)])

In [59]:
pipe = Pipeline([('preprocess', ct),
                      ('model',
                       LogisticRegression(max_iter = 10000))])
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE',
                                                   'VEHICLE_TYPE'])])),
                ('model', LogisticRegression(max_iter=10000))])

In [60]:
df2['target_mapped'].value_counts()

reckless driving                         119561
missed or misunderstood traffic signs     57137
issue with the driver or vehicle          21942
external variable                         17381
Name: target_mapped, dtype: int64

In [61]:
df3 = df2[['MAKE', 'MOST_SEVERE_INJURY', 'DAMAGE', 'VEHICLE_TYPE', 'target_mapped']]

In [62]:
df3['target_mapped'].value_counts()

reckless driving                         119561
missed or misunderstood traffic signs     57137
issue with the driver or vehicle          21942
external variable                         17381
Name: target_mapped, dtype: int64

In [63]:
X = df3.drop('target_mapped', axis=1)
y = df3['target_mapped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipe.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['MAKE', 'MOST_SEVERE_INJURY',
                                                   'DAMAGE',
                                                   'VEHICLE_TYPE'])])),
                ('model', LogisticRegression(max_iter=10000))])

In [64]:
print(pipe.score(X_train,y_train))
pipe.score(X_test,y_test)

0.5543873252476622


0.5576900821664159

In [65]:
preds = pipe.predict(X_test)

confusion_matrix(y_test, preds)

array([[    0,     0,   221,  3182],
       [    0,     7,   459,  3847],
       [    0,     2,  1460,  9969],
       [    0,     5,  1425, 22628]])

# More cleaning and feature engineering

In [66]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216021 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    216021 non-null  object 
 1   SEX                            216021 non-null  object 
 2   EJECTION                       216021 non-null  object 
 3   INJURY_CLASSIFICATION          216021 non-null  object 
 4   DRIVER_VISION                  216021 non-null  object 
 5   PHYSICAL_CONDITION             216021 non-null  object 
 6   POSTED_SPEED_LIMIT             216021 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         216021 non-null  object 
 8   DEVICE_CONDITION               216021 non-null  object 
 9   WEATHER_CONDITION              216021 non-null  object 
 10  LIGHTING_CONDITION             216021 non-null  object 
 11  FIRST_CRASH_TYPE               216021 non-null  object 
 12  TRAFFICWAY_TYPE              

In [67]:
df2.columns

Index(['PERSON_TYPE', 'SEX', 'EJECTION', 'INJURY_CLASSIFICATION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION', 'POSTED_SPEED_LIMIT',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION', 'WEATHER_CONDITION',
       'LIGHTING_CONDITION', 'FIRST_CRASH_TYPE', 'TRAFFICWAY_TYPE',
       'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT', 'REPORT_TYPE',
       'CRASH_TYPE', 'DAMAGE', 'DATE_POLICE_NOTIFIED',
       'PRIM_CONTRIBUTORY_CAUSE', 'STREET_NO', 'STREET_DIRECTION',
       'STREET_NAME', 'BEAT_OF_OCCURRENCE', 'NUM_UNITS', 'MOST_SEVERE_INJURY',
       'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
       'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
       'INJURIES_NO_INDICATION', 'INJURIES_UNKNOWN', 'CRASH_HOUR',
       'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'LATITUDE', 'LONGITUDE', 'LOCATION',
       'CRASH_DATE', 'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
       'VEHICLE_YEAR', 'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRE

In [68]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 216021 entries, 12 to 731904
Data columns (total 58 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   PERSON_TYPE                    216021 non-null  object 
 1   SEX                            216021 non-null  object 
 2   EJECTION                       216021 non-null  object 
 3   INJURY_CLASSIFICATION          216021 non-null  object 
 4   DRIVER_VISION                  216021 non-null  object 
 5   PHYSICAL_CONDITION             216021 non-null  object 
 6   POSTED_SPEED_LIMIT             216021 non-null  int64  
 7   TRAFFIC_CONTROL_DEVICE         216021 non-null  object 
 8   DEVICE_CONDITION               216021 non-null  object 
 9   WEATHER_CONDITION              216021 non-null  object 
 10  LIGHTING_CONDITION             216021 non-null  object 
 11  FIRST_CRASH_TYPE               216021 non-null  object 
 12  TRAFFICWAY_TYPE              

In [69]:
df2['UNIT_TYPE'].value_counts()

DRIVER                 214012
PARKED                   1814
DRIVERLESS                174
NON-CONTACT VEHICLE        12
DISABLED VEHICLE            9
Name: UNIT_TYPE, dtype: int64

In [70]:
for column in df2[cat_cols].columns:
    print(f"Column: {column}")
    print(df2[column].value_counts())
    print("\n")

Column: MAKE
CHEVROLET                            27642
TOYOTA                               26423
FORD                                 25523
NISSAN                               18479
HONDA                                17352
                                     ...  
INTERNATIONAL COACH MANUFACTURING       56
MOTOR COACH INDUSTRIES, INC.            55
MASERATI                                55
BLUE BIRD BODY CO.                      52
ICL CARGO TRAILER                       52
Name: MAKE, Length: 67, dtype: int64


Column: MOST_SEVERE_INJURY
NO INDICATION OF INJURY     180649
NONINCAPACITATING INJURY     20033
REPORTED, NOT EVIDENT        10929
INCAPACITATING INJURY         4216
FATAL                          194
Name: MOST_SEVERE_INJURY, dtype: int64


Column: DAMAGE
OVER $1,500      143628
$501 - $1,500     51419
$500 OR LESS      20974
Name: DAMAGE, dtype: int64


Column: VEHICLE_TYPE
PASSENGER                         147477
SPORT UTILITY VEHICLE (SUV)        31226
VAN/MINI-VAN 

In [71]:
for column in df2[num_cols].columns:
    print(f"Column: {column}")
    print(df2[column].value_counts())
    print("\n")

NameError: name 'num_cols' is not defined

In [72]:
target_dict

{'reckless driving': ['FOLLOWING TOO CLOSELY',
  'IMPROPER OVERTAKING/PASSING',
  'EXCEEDING AUTHORIZED SPEED LIMIT',
  'IMPROPER TURNING/NO SIGNAL',
  'IMPROPER BACKING',
  'IMPROPER LANE USAGE',
  'FAILING TO REDUCE SPEED TO AVOID CRASH',
  'OPERATING VEHICLE IN ERRATIC, RECKLESS, CARELESS, NEGLIGENT OR AGGRESSIVE MANNER',
  'TURNING RIGHT ON RED',
  'DRIVING ON WRONG SIDE/WRONG WAY',
  'CELL PHONE USE OTHER THAN TEXTING',
  'TEXTING',
  'DISTRACTION - OTHER ELECTRONIC DEVICE (NAVIGATION DEVICE, DVD PLAYER, ETC.)',
  'DISTRACTION - FROM INSIDE VEHICLE'],
 'missed or misunderstood traffic signs': ['DISREGARDING YIELD SIGN',
  'FAILING TO YIELD RIGHT-OF-WAY',
  'DISREGARDING TRAFFIC SIGNALS',
  'DISREGARDING OTHER TRAFFIC SIGNS',
  'DISREGARDING ROAD MARKINGS',
  'DISREGARDING STOP SIGN'],
 'issue with the driver or vehicle': ['DRIVING SKILLS/KNOWLEDGE/EXPERIENCE',
  'PHYSICAL CONDITION OF DRIVER',
  'EQUIPMENT - VEHICLE CONDITION',
  'HAD BEEN DRINKING (USE WHEN ARREST IS NOT MADE)',


In [73]:
df2['target_mapped'].value_counts()

reckless driving                         119561
missed or misunderstood traffic signs     57137
issue with the driver or vehicle          21942
external variable                         17381
Name: target_mapped, dtype: int64

# Now lets go back to modeling

In [74]:
numeric_pipeline = Pipeline([('numnorm', StandardScaler())])

#ordinal_pipeline = Pipeline([
  #  ('ordenc', OrdinalEncoder(categories = [columns])),
  #  ('ordnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(sparse = False, handle_unknown = 'ignore')), 
    ('onehotnorm', MaxAbsScaler())])

In [75]:
cat_cols = ['PERSON_TYPE', 'SEX', 'EJECTION', 'INJURY_CLASSIFICATION',
       'DRIVER_VISION', 'PHYSICAL_CONDITION',
       'TRAFFIC_CONTROL_DEVICE', 'DEVICE_CONDITION',
       'WEATHER_CONDITION', 'LIGHTING_CONDITION', 
       'TRAFFICWAY_TYPE', 'ALIGNMENT', 'ROADWAY_SURFACE_COND', 'ROAD_DEFECT',
       'REPORT_TYPE', 'CRASH_TYPE', 'DAMAGE', 
        'STREET_DIRECTION', 
       'STREET_NAME', 'MOST_SEVERE_INJURY',
        'UNIT_TYPE', 'MAKE', 'MODEL', 'LIC_PLATE_STATE',
        'VEHICLE_DEFECT', 'VEHICLE_TYPE', 'VEHICLE_USE',
       'TRAVEL_DIRECTION', 'MANEUVER', 'FIRST_CONTACT_POINT',
       'BAC_RESULT', 'SAFETY_EQUIPMENT', 'AIRBAG_DEPLOYED']
num_cols = ['AGE', 'STREET_NO', 'VEHICLE_YEAR', 'CRASH_HOUR',
            'CRASH_DAY_OF_WEEK', 'CRASH_MONTH', 'BEAT_OF_OCCURRENCE', 'POSTED_SPEED_LIMIT',
            'LATITUDE', 'LONGITUDE', 'OCCUPANT_CNT',
            'INJURIES_TOTAL', 'INJURIES_FATAL', 'INJURIES_INCAPACITATING',
            'INJURIES_NON_INCAPACITATING', 'INJURIES_REPORTED_NOT_EVIDENT',
            'INJURIES_NO_INDICATION', 'NUM_UNITS', 'INJURIES_UNKNOWN']

#got rid of 'FIRST_CRASH_TYPE' due to data leakage concerns

In [76]:
ct = ColumnTransformer(
    [ ("nominalpipe", nominal_pipeline, cat_cols),
      ("numericpipe", numeric_pipeline, num_cols)
    ])

In [77]:
X.shape

(216021, 4)

In [78]:
X = df2.drop('target_mapped', axis=1)
y = df2['target_mapped']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [79]:
pipe = Pipeline([('preprocess', ct), 
                     ('model',
                        LogisticRegression(solver = 'liblinear'))])
pipe.fit(X_train, y_train)
pipe

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['PERSON_TYPE', 'SEX',
                                                   'EJECTION',
                                                   'INJURY_CLASSIFICATION',
                                                   'DRIVER_VISION',
                                                   'PHYSICAL_CONDITION',
                                                   'TRAFFIC_CONTROL_DEVICE',
                        

In [80]:
print(pipe.score(X_train,y_train))
pipe.score(X_test,y_test)

0.6863021942412739


0.6784399953709062

In [81]:
# Get predictions
y_pred = pipe.predict(X_test)

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report (includes precision, recall, and F1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC Score
# Need probabilities of the positive class for ROC AUC Score
y_pred_proba = pipe.predict_proba(X_test)[:,1]
print("ROC AUC Score:")
print(roc_auc_score(y_test, y_pred_proba))

Confusion Matrix:
[[ 1297    74   478  1554]
 [  157  1542   472  2142]
 [  225   123  5879  5204]
 [  519   566  2379 20594]]
Classification Report:
                                       precision    recall  f1-score   support

                    external variable       0.59      0.38      0.46      3403
     issue with the driver or vehicle       0.67      0.36      0.47      4313
missed or misunderstood traffic signs       0.64      0.51      0.57     11431
                     reckless driving       0.70      0.86      0.77     24058

                             accuracy                           0.68     43205
                            macro avg       0.65      0.53      0.57     43205
                         weighted avg       0.67      0.68      0.66     43205

ROC AUC Score:


ValueError: multi_class must be in ('ovo', 'ovr')

In [82]:
# Get predictions
y_pred = pipe2.predict(X_test)

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report (includes precision, recall, and F1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC Score
# Need probabilities of the positive class for ROC AUC Score
y_pred_proba = pipe2.predict_proba(X_test)
print("ROC AUC Score:")
print(roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted'))

NameError: name 'pipe2' is not defined

In [83]:
pipe2 = Pipeline([
     ('preprocess', ct), 
     ('model', RandomForestClassifier(max_depth=20))
 ])

pipe2.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['PERSON_TYPE', 'SEX',
                                                   'EJECTION',
                                                   'INJURY_CLASSIFICATION',
                                                   'DRIVER_VISION',
                                                   'PHYSICAL_CONDITION',
                                                   'TRAFFIC_CONTROL_DEVICE',
                        

In [84]:
print(pipe2.score(X_train,y_train))
pipe2.score(X_test,y_test)

0.6659857883529303


0.6333063302858466

In [85]:
# Get predictions
y_pred = pipe2.predict(X_test)

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report (includes precision, recall, and F1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC Score
# Need probabilities of the positive class for ROC AUC Score
y_pred_proba = pipe.predict_proba(X_test)[:,1]
print("ROC AUC Score:")
print(roc_auc_score(y_test, y_pred_proba))

Confusion Matrix:
[[  488     4   274  2637]
 [   21   550   206  3536]
 [    5    19  3468  7939]
 [   38   120  1044 22856]]
Classification Report:
                                       precision    recall  f1-score   support

                    external variable       0.88      0.14      0.25      3403
     issue with the driver or vehicle       0.79      0.13      0.22      4313
missed or misunderstood traffic signs       0.69      0.30      0.42     11431
                     reckless driving       0.62      0.95      0.75     24058

                             accuracy                           0.63     43205
                            macro avg       0.75      0.38      0.41     43205
                         weighted avg       0.68      0.63      0.57     43205

ROC AUC Score:


ValueError: multi_class must be in ('ovo', 'ovr')

In [86]:
# Get the feature importances
feature_importances = pipe2.named_steps['model'].feature_importances_

importances = pd.Series(feature_importances, index=all_feature_names)
importances = importances.abs().sort_values(ascending=False)
importances.head(60)


NameError: name 'all_feature_names' is not defined

In [87]:
pipe1 = Pipeline([('preprocess', ct), 
                 ('model', XGBClassifier(eval_metric='logloss'))])

pipe1.fit(X_train, y_train)


Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('nominalpipe',
                                                  Pipeline(steps=[('onehotenc',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False)),
                                                                  ('onehotnorm',
                                                                   MaxAbsScaler())]),
                                                  ['PERSON_TYPE', 'SEX',
                                                   'EJECTION',
                                                   'INJURY_CLASSIFICATION',
                                                   'DRIVER_VISION',
                                                   'PHYSICAL_CONDITION',
                                                   'TRAFFIC_CONTROL_DEVICE',
                        

In [88]:
print(pipe1.score(X_train,y_train))
pipe1.score(X_test,y_test)

0.7300307841866494


0.7033445203101493

In [ ]:
# Get predictions
y_pred = pipe1.predict(X_test)

# Confusion Matrix
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Classification Report (includes precision, recall, and F1-score)
print("Classification Report:")
print(classification_report(y_test, y_pred))

# ROC AUC Score
# Need probabilities of the positive class for ROC AUC Score
y_pred_proba = pipe1.predict_proba(X_test)
print("ROC AUC Score:")
print(roc_auc_score(y_test, y_pred_proba, multi_class='ovr', average='weighted'))

In [ ]:
# Get the feature importances
feature_importances = pipe1.named_steps['model'].feature_importances_

importances = pd.Series(feature_importances, index=all_feature_names)
importances = importances.abs().sort_values(ascending=False)
importances.head(7)


In [ ]:
import multiprocessing

num_cores = multiprocessing.cpu_count()
print(num_cores)
multiprocessing.gpu_count()

In [ ]:
df2['FIRST_CRASH_TYPE'].value_counts()

# Visualizations

In [ ]:
df2['target_mapped'].value_counts()

In [ ]:


# Get the value counts
value_counts = df2['target_mapped'].value_counts()

# Create a figure and axis
fig, ax = plt.subplots(figsize=(10, 8))

# Generate a bar plot
ax.bar(value_counts.index, value_counts.values, color=['blue', 'green', 'red', 'purple'])

# Add labels and title
ax.set_ylabel('Number of Accidents')
ax.set_xlabel('Reason for Accident')
plt.title('Number of Accidents by Reason')
plt.xticks(rotation=45, ha='right')  # Rotate the x-labels for better readability

# Show the plot
plt.show()


In [ ]:
# Set seaborn aesthetic parameters to defaults
sns.set_theme()

# Set a dark grid background
sns.set_style("darkgrid")

# Create a color palette
colors = ["#3498db", "#2ecc71", "#e74c3c", "#9b59b6"]

# Increase the figure size
plt.figure(figsize=(10, 8))

# Get the value counts
value_counts = df2['target_mapped'].value_counts()

# Create a barplot
sns.barplot(y=value_counts.index, x=value_counts.values, palette=colors)

# Add labels and title
plt.xlabel('Number of Accidents', fontsize=14)
plt.title('Number of Accidents by Primary Reason', fontsize=16)

# Show the plot
plt.show()


In [ ]:

# Set seaborn aesthetic parameters to defaults
sns.set_theme()

# Set a specific background style
plt.style.use('fivethirtyeight')


# Increase the figure size
plt.figure(figsize=(10, 8))

# Get the value counts
value_counts = df2['target_mapped'].value_counts()

# Create a barplot
sns.barplot(y=value_counts.index, x=value_counts.values, palette="pastel")

# Add labels and title
plt.xlabel('Number of Accidents', fontsize=16)
plt.title('Number of Accidents by Reason', fontsize=20)

# Change the font size of the labels for each bar
y_ticks = plt.yticks()
plt.setp(y_ticks[1], size=14)

# Show the plot
plt.show()


In [ ]:
target_dict

In [216]:
X['TRAFFIC_CONTROL_DEVICE'].value_counts()

NO CONTROLS                 108929
TRAFFIC SIGNAL               71479
STOP SIGN/FLASHER            26101
UNKNOWN                       3654
OTHER                         1518
LANE USE MARKING               542
YIELD                          437
OTHER REG. SIGN                283
OTHER WARNING SIGN             274
RAILROAD CROSSING GATE         157
PEDESTRIAN CROSSING SIGN       138
DELINEATORS                    124
FLASHING CONTROL SIGNAL         90
POLICE/FLAGMAN                  83
SCHOOL ZONE                     74
OTHER RAILROAD CROSSING         62
RR CROSSING SIGN                27
NO PASSING                      14
BICYCLE CROSSING SIGN            2
Name: TRAFFIC_CONTROL_DEVICE, dtype: int64

In [222]:
df2[df2['TRAFFIC_CONTROL_DEVICE'] == 'STOP SIGN/FLASHER']['target_mapped'].value_counts()


missed or misunderstood traffic signs    14317
reckless driving                          8267
external variable                         1954
issue with the driver or vehicle          1563
Name: target_mapped, dtype: int64

In [220]:
df2[df2['PHYSICAL_CONDITION'] == 'IMPAIRED - ALCOHOL']['target_mapped'].value_counts()

issue with the driver or vehicle         2399
reckless driving                          640
missed or misunderstood traffic signs     191
external variable                          17
Name: target_mapped, dtype: int64

In [221]:
df2[df2['MANEUVER'] == 'ENTERING TRAFFIC LANE FROM PARKING']['target_mapped'].value_counts()

missed or misunderstood traffic signs    3425
reckless driving                          977
issue with the driver or vehicle          398
external variable                         135
Name: target_mapped, dtype: int64

In [228]:
df2[df2['WEATHER_CONDITION'] == 'SNOW']['target_mapped'].value_counts()

reckless driving                         3579
external variable                        3405
missed or misunderstood traffic signs    1580
issue with the driver or vehicle          631
Name: target_mapped, dtype: int64